In [27]:
import pandas as pd
reviews = pd.read_csv('../Syntetic_reviews/reviews_all_aumented_iter1.csv').dropna()



array(['Material and Quality', 'Fit and Comfort'], dtype=object)

In [76]:

import numpy as np
from scipy.spatial import distance
from collections import defaultdict
from typing import List, Tuple
import spacy
from fast_sentence_transformers import FastSentenceTransformer as SentenceTransformer


import numpy as np


class VectorDatabase:
    def __init__(self,nlp,model):
        self.vectors = {}
        self.nlp = nlp
        self.model = model
        self.very_similar = 0.5
        self.similar = 0.5
        

    def split_sentences(self, text):
        text = text.replace(',','.').replace('and','.').replace('but','.')
        doc = self.nlp(text, disable=["ner"])
        roots = [token  for token in doc if token.dep_ == "ROOT" ]
    
        texts = []
        for root in roots:
            token_list = [e.i for e in root.subtree]
            token_list = list(dict.fromkeys(token_list))
            token_list.sort()
            text = ' '.join([doc[i].text for i in token_list ])
            texts.append(text.lower().strip())
            
        return texts#text.replace(',','.').replace('but','.').split('.')


    def insert(self, sentence: str, polarity: int, type: str) -> None:
        model = self.model
        #embeddings = model.encode(sentence)
        embeddings = list(model.encode([sentence])[0])
        key = len(self.vectors) + 1
        self.vectors[key] = {'text': sentence,
                             'polarity': polarity,
                             'type': type,
                             'vector': embeddings}

    def search(self, query: str):
        model = self.model
        #query_vector = model.encode(query)
        query_vector = list(model.encode([query])[0])
        
        similarities = [(key, value['text'],distance.cosine(query_vector, value['vector']),value['polarity'],value['type']) for key, value in self.vectors.items()]
        

        aux = pd.DataFrame(similarities)
        aux.columns = ['index_db','text','similarity','polarity','topic']
        aux = aux.sort_values(by=['similarity']).reset_index(drop=True).reset_index()

        #aux = aux.reset_index().query('index<20 or similarity<0.7').query('similarity<1')[['index','topic']].groupby(['topic']).count()
        
        aux = aux.query('index<=5')
        #aux = aux.query('similarity <={}'.format(self.very_similar))

        aux = aux.query('similarity <={}'.format(self.similar))
        
        aux = aux[['index','topic']].groupby(['topic']).count()
        
        
        aux['index2'] = aux['index']/aux['index'].sum()
     
       # aux = aux.query('index2>0.4')
        

        #aux = aux.sort_values(by='index', ascending=False)
                
        return  list(aux.index.values)

    def long_search(self, query: str):
        topics = []
        for str in self.split_sentences(query):
            topics_this = self.search(str)
            if len(topics_this)>0:
                mini_df = pd.DataFrame(topics_this)
                mini_df.columns = ['topic']
                mini_df['review'] = query
                mini_df['sub_review'] = str
                topics.append(mini_df)
        if len(topics)>0:
            
            aux = pd.concat(topics)
            #aux ['stars'] = [int(self.sentiment_pipe(str)[0]['label'][0]) for str in aux.sub_review]
        else:
            aux = None
            
        return  aux

    def set_th(self):
        data = pd.DataFrame(self.vectors).transpose()


        n = data.shape[0]
        if(n>1000):
            n = min(max(int(n*0.1),1000),n)
            data = data.sample(n)
        print('Total size to compute the is {}'.format(data.shape[0]))

        same_type_similarity = []
        
        same_type_top_similarity = []
        
        for i in range(len(data.vector)):
            
        
            vectors = data.vector
            vector = vectors.values[i]
            aux = pd.DataFrame(
                [distance.cosine(vector, vectors[i]) for i in vectors.keys()]
            )
            
            aux.columns = ['similarity']
            
            aux['topic'] = data.type.values
            
            topic_review = data.type.values[i]
             
            same_type_similarity.append(np.percentile(aux.query(f'topic=="{topic_review}"').similarity,25))
        
            same_type_top_similarity.append(np.percentile(aux.query(f'topic=="{topic_review}"').similarity,25))
        
        self.very_similar = np.percentile(same_type_top_similarity,50)
        self.similar = np.percentile(same_type_similarity,25)




import pandas as pd

#reviews = pd.read_csv('../Syntetic_reviews/reviews_all_aumented_iter1.csv').dropna()
#reviews = pd.read_csv('../Syntetic_reviews/reviews_all.csv').dropna()
#reviews = pd.read_csv('../Syntetic_reviews/openai_syn.csv').dropna() 


import json
 
# Opening JSON file
f = open('../Syntetic_reviews/syn_reviews.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
reviews = []

for key in data:
    rev = pd.DataFrame(data[key])
    rev.columns = ['Review']
    rev['Topic'] = key
    reviews.append(rev)

reviews = pd.concat(reviews)

#reviews = pd.read_csv('../Syntetic_reviews/openai_syn.csv').dropna() 
reviews = pd.read_csv('../Syntetic_reviews/reviews_all_aumented_iter1.csv').dropna()

oks = ['Fit and Comfort','Material and Quality']
#reviews = reviews.query('Topic in @oks')
reviews.Topic.unique()

manual = pd.read_csv('../Syntetic_reviews/manual_tags.csv')


from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

nlp = spacy.load("en_core_web_lg")


vector_db = VectorDatabase(nlp, model)
print('uploading vectors to DB')
for index, row in reviews.iterrows():
    vector_db.insert(row['Review'],None,row['Topic'])


vector_db.set_th()


uploading vectors to DB


KeyboardInterrupt: 

In [119]:


model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
query_vector = list(model.encode('''How is the quality of the materials?'''))
#query_vector = list(model.encode('Is it easy to use?'))



review = '''

Really I'm amazed by how good is the leather

'''
ans = list(model.encode(review))

distance.cosine(query_vector, ans)

0.3938968777656555

0.6024659872055054

In [66]:
guess = [] 
recalls = []
precisions = []
for index, row in manual.iterrows():

    aux = vector_db.long_search(row['Review'])
    my_guess = []
    if aux is not None:
        my_guess = list(aux.topic.unique())
    real = list(row.dropna().index)
    if len(real)>1:
        real = real[1:]
        real = list(row.dropna().index)[1:] 
        recall = pd.DataFrame(real)
        recall.columns = ['topic']
        recall['value'] = [ t in my_guess for t in real]
        recalls.append(recall)
    
    if len(my_guess)>0:
        precision = pd.DataFrame(my_guess)
        precision.columns = ['topic']
        precision['value'] = [ t in real for t in my_guess]
        precisions.append(precision)

recall = pd.concat(recalls).groupby(['topic']).mean().reset_index()
precision = pd.concat(precisions).groupby(['topic']).mean().reset_index()

metrics = recall.merge(precision, on = ['topic'], how = 'inner')

metrics.columns = ['topic','recall','precision']

metrics

,topic,recall,precision
0,Fit and Comfort,0.875000,0.717949
1,Longevity,0.250000,0.017544
2,Material and Quality,0.815789,0.436620
3,Packaging and Presentation,1.000000,0.021739
4,Price and Value,0.937500,0.187500
5,User Experience,0.270270,0.158730
6,Versatility,0.461538,0.120000


In [49]:
pd.concat(precisions).query('topic in @oks').value.mean()

0.5797101449275363